In [9]:
from make_benchmarks import *
import os

In [2]:
work_dir = "/data/rashika/CAFA4/"


t0_mapped_path = work_dir + "mapped/t0.csv"
t1_mapped_path = work_dir + "mapped/t1.csv"

#t0_mapped_path = work_dir + "mapped/t1_test.csv"
#t1_mapped_path = work_dir + "mapped/t0_test.csv"

t0_ont_file = '/data/rashika/CAFA4/uniprot/go_2019_12_09/go-basic.obo' 
t0_ont_graph = clean_ontology_edges(obonet.read_obo(t0_ont_file))     

t1_ont_file = "/data/rashika/CAFA4/uniprot/go_2024_03_28/go-basic.obo"
t1_ont_graph = clean_ontology_edges(obonet.read_obo(t1_ont_file)) 

t_minus_ont_file =  "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(t_minus_ont_file))

c3_t_minus_ont_file =  "/data/rashika/CAFA4/head2head/CAFA3/go.obo"
c3_t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(c3_t_minus_ont_file))

c2_t_minus_ont_file =  "/data/rashika/CAFA4/head2head/CAFA2/go.obo"
c2_t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(c2_t_minus_ont_file))


In [3]:
# roots is a list of the root terms
def create_bm_lists_h2h(t0_file, t1_file, t0_ont_graph, t1_ont_graph, t1_minus_ont_graph,roots, c2_t_minus_1_ont_graph, c3_t_minus_1_ont_graph, BM_path = "/data/rashika/CAFA4/head2head/eval/BM_GO/"):

    #Prop t0 and t1 in their respective ontologies
    t0_prop = process_raw_annot(t0_file, t0_ont_graph, roots)
    t1_prop = process_raw_annot(t1_file, t1_ont_graph, roots)
    
    # Keep terms common in t0_ont and t1_ont
    t0_common, t1_common =  keep_common_go_terms(t0_prop, t1_prop, t0_ont_graph, t1_ont_graph)
    print(len(t0_common), len(t1_common))
    
    # Keep terms common in (t0_ont, t1_ont) and (c2_t_minus_1,, c3_tminus_1) 
    t0_common, t1_common =  keep_common_go_terms(t0_common, t1_common, c2_t_minus_1_ont_graph, c2_t_minus_1_ont_graph)
    print(len(t0_common), len(t1_common))

    # Propagate back in the t(-1) ontology
    subontologies = {aspect: fetch_aspect(t1_minus_ont_graph, roots[aspect]) for aspect in roots}
    t0_eval = propagate_terms(t0_common, subontologies)
    t1_eval = propagate_terms(t1_common, subontologies)

    # Convert the eval Dfs into annotation lists
    t0_annot_list = get_annot_list(t0_eval)
    t1_annot_list = get_annot_list(t1_eval)
    
    # get baselines
    get_baselines(t0_annot_list, t1_annot_list,  BM_path)

def keep_common_go_terms(t0, t1, t0_ont, t1_ont):
    
    # Get the three subontologies
    roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}
    t0_subont = {aspect: fetch_aspect(t0_ont, roots[aspect]) for aspect in roots}
    t1_subont = {aspect: fetch_aspect(t1_ont, roots[aspect]) for aspect in roots}
    
    common_terms = {aspect: set(t1_subont[aspect].nodes).intersection(set(t0_subont[aspect].nodes)) for aspect in ['BPO', 'CCO', 'MFO']}
    
    
    t0_common = delete_extra_go_terms(t0, common_terms)
    t1_common = delete_extra_go_terms(t1, common_terms)
    return t0_common, t1_common

In [4]:
roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}
#create_bm_lists_h2h(t0_mapped_path,t1_mapped_path, t0_ont_graph, t1_ont_graph, t_minus_1_ont_graph,roots, c2_t_minus_1_ont_graph, c3_t_minus_1_ont_graph, BM_path = "/data/rashika/CAFA4/head2head/eval/BM_GO/")

3178947 3187762
2967460 2955986
/data/rashika/CAFA4/head2head/eval/BM_GO/bpo_all_type1.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/cco_all_type1.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/mfo_all_type1.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/bpo_all_type2.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/cco_all_type2.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/mfo_all_type2.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/bpo_all_type3.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/cco_all_type3.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/mfo_all_type3.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/bpo_all_type12.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/cco_all_type12.txt
/data/rashika/CAFA4/head2head/eval/BM_GO/mfo_all_type12.txt


### Use the created benchmarks to map to the benchmarks that are in all the CAFAs

In [11]:
h2h_proteins = pd.read_csv('/data/common/CAFA4/h2h/H2H_CAFA4.tsv', sep = '\t', header = 0)
display(h2h_proteins)
BM_path = "/data/rashika/CAFA4/head2head/eval/BM_GO/"

,AC ID,Sequence,Taxon ID,CAFA_ID_2,CAFA_ID_3,CAFA_ID_4,Entry_ID_2,Entry_ID_3,Entry_ID_4
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...,10090,T100900000001,T100900000001,T100900000001,1433B_MOUSE,1433B_MOUSE,1433B_MOUSE
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,10090,T100900000002,T100900000002,T100900000002,1433E_MOUSE,1433E_MOUSE,1433E_MOUSE
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,10090,T100900000003,T100900000003,T100900000003,1433F_MOUSE,1433F_MOUSE,1433F_MOUSE
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,10090,T100900000004,T100900000004,T100900000004,1433G_MOUSE,1433G_MOUSE,1433G_MOUSE
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...,10090,T100900000005,T100900000005,T100900000005,1433S_MOUSE,1433S_MOUSE,1433S_MOUSE
...,...,...,...,...,...,...,...,...,...
76514,Q8ZNV7,MTSLVSLENVSVSFGQRRVLSDVSLELSPGKILTLLGPNGAGKSTL...,99287,T992870001767,T992870001785,T992870001793,ZNUC_SALTY,ZNUC_SALTY,ZNUC_SALTY
76515,Q9L9I0,MKRNNKSAIALIALSLLALSSGAAFAGHHWGNNDGMWQQGGSPLTT...,99287,T992870001768,T992870001786,T992870001794,ZRAP_SALTY,ZRAP_SALTY,ZRAP_SALTY
76516,P25852,MIRGKIDILVVDDDVSHCTILQALLRGWGYNVALAYSGHDALAQVR...,99287,T992870001769,T992870001787,T992870001795,ZRAR_SALTY,ZRAR_SALTY,ZRAR_SALTY
76517,P37461,MSFIRLHKDAAATWLSRLLPAAIFILVGLFSIMVIRDYGRESAAAR...,99287,T992870001770,T992870001788,T992870001796,ZRAS_SALTY,ZRAS_SALTY,ZRAS_SALTY


In [37]:
BM_list = os.listdir(BM_path)
mapping_4_to_3 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_3']))
mapping_4_to_2 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_2']))
mapping_4_to_4 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_4']))
for bm_file in BM_list:
    print(bm_file)
    bm_df = pd.read_csv(BM_path+bm_file, header = None, sep = '\t')
    bm_df.columns = ['ID', 'GO ID', 'aspect']
    
    # Shortlist the proteins to the h2h common proteins
    bm_df = bm_df[bm_df['ID'].isin(h2h_proteins['CAFA_ID_4'])].copy()
    
    # Map to IDs from respective CAFAs
    bm_df['CAFA3_ID'] = bm_df['ID'].map(mapping_4_to_3)
    bm_df['CAFA2_ID'] = bm_df['ID'].map(mapping_4_to_2)
    bm_df['CAFA4_ID'] = bm_df['ID'].map(mapping_4_to_4)
    
    # Get the benchmark copy for respective CAFA
    bm_df_cafa4 = bm_df.loc[:, ["CAFA4_ID", "GO ID", "aspect"]].copy()
    bm_df_cafa3 = bm_df.loc[:, ["CAFA3_ID", "GO ID", "aspect"]].copy()
    bm_df_cafa2 = bm_df.loc[:, ["CAFA2_ID", "GO ID", "aspect"]].copy()
    
    # Make the directory to save the mapped benchmarks for each cafa
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4')
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa3')
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa2')
    bm_df_cafa4.to_csv('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/' + bm_file, sep = '\t', header = None, index = False)
    bm_df_cafa3.to_csv('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa3/' + bm_file, sep = '\t', header = None, index = False)
    bm_df_cafa2.to_csv('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa2/' + bm_file, sep = '\t', header = None, index = False)
    
    display(bm_df)

bpo_all_type1.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T100900005994,GO:0002831,BPO,T100900005984,T100900005960,T100900005994
1,T100900005994,GO:0043207,BPO,T100900005984,T100900005960,T100900005994
2,T100900005994,GO:0009607,BPO,T100900005984,T100900005960,T100900005994
3,T100900005994,GO:0002684,BPO,T100900005984,T100900005960,T100900005994
4,T100900005994,GO:0031347,BPO,T100900005984,T100900005960,T100900005994
...,...,...,...,...,...,...
37861,T100900005813,GO:0051049,BPO,T100900005802,T100900005776,T100900005813
37862,T100900005813,GO:0010646,BPO,T100900005802,T100900005776,T100900005813
37863,T100900005813,GO:0098693,BPO,T100900005802,T100900005776,T100900005813
37864,T100900005813,GO:0065008,BPO,T100900005802,T100900005776,T100900005813


cco_all_type1.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T100900005994,GO:0071944,CCO,T100900005984,T100900005960,T100900005994
1,T100900005994,GO:0110165,CCO,T100900005984,T100900005960,T100900005994
2,T100900005994,GO:0009898,CCO,T100900005984,T100900005960,T100900005994
3,T100900005994,GO:0098562,CCO,T100900005984,T100900005960,T100900005994
4,T100900005994,GO:0098552,CCO,T100900005984,T100900005960,T100900005994
...,...,...,...,...,...,...
13117,T100900005813,GO:0098588,CCO,T100900005802,T100900005776,T100900005813
13118,T100900005813,GO:0070382,CCO,T100900005802,T100900005776,T100900005813
13119,T100900005813,GO:0099503,CCO,T100900005802,T100900005776,T100900005813
13120,T100900005813,GO:0005886,CCO,T100900005802,T100900005776,T100900005813


mfo_all_type1.txt


...

,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T100900005994,GO:0002831,BPO,T100900005984,T100900005960,T100900005994
1,T100900005994,GO:0043207,BPO,T100900005984,T100900005960,T100900005994
2,T100900005994,GO:0009607,BPO,T100900005984,T100900005960,T100900005994
3,T100900005994,GO:0002684,BPO,T100900005984,T100900005960,T100900005994
4,T100900005994,GO:0031347,BPO,T100900005984,T100900005960,T100900005994
...,...,...,...,...,...,...
77467,T96060017546,GO:0097435,BPO,T96060017425,T96060017412,T96060017546
77468,T96060017546,GO:0009987,BPO,T96060017425,T96060017412,T96060017546
77469,T96060017546,GO:0022607,BPO,T96060017425,T96060017412,T96060017546
77470,T96060017546,GO:0071840,BPO,T96060017425,T96060017412,T96060017546


cco_all_type12.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T100900005994,GO:0071944,CCO,T100900005984,T100900005960,T100900005994
1,T100900005994,GO:0110165,CCO,T100900005984,T100900005960,T100900005994
2,T100900005994,GO:0009898,CCO,T100900005984,T100900005960,T100900005994
3,T100900005994,GO:0098562,CCO,T100900005984,T100900005960,T100900005994
4,T100900005994,GO:0098552,CCO,T100900005984,T100900005960,T100900005994
...,...,...,...,...,...,...
24707,T37020011715,GO:0000151,CCO,T37020010883,T37020008969,T37020011715
24708,T37020011715,GO:0005575,CCO,T37020010883,T37020008969,T37020011715
24741,T833330000795,GO:0032991,CCO,T833330000769,T833330000748,T833330000795
24742,T833330000795,GO:0005575,CCO,T833330000769,T833330000748,T833330000795


mfo_all_type12.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
5,T100900005994,GO:0005488,MFO,T100900005984,T100900005960,T100900005994
6,T100900005994,GO:0019992,MFO,T100900005984,T100900005960,T100900005994
7,T100900005994,GO:0008289,MFO,T100900005984,T100900005960,T100900005994
8,T100900005994,GO:0003674,MFO,T100900005984,T100900005960,T100900005994
9,T96060005494,GO:0097159,MFO,T96060005488,T96060005519,T96060005494
...,...,...,...,...,...,...
27093,T446890002398,GO:0003674,MFO,T446890002390,T446890002381,T446890002398
27094,T446890002398,GO:0005515,MFO,T446890002390,T446890002381,T446890002398
27095,T100900002681,GO:0005488,MFO,T100900002651,T100900002609,T100900002681
27096,T100900002681,GO:0005515,MFO,T100900002651,T100900002609,T100900002681


In [35]:
# Test a protein mapping
h2h_proteins.loc[h2h_proteins["CAFA_ID_4"] == "T100900005994"]

,AC ID,Sequence,Taxon ID,CAFA_ID_2,CAFA_ID_3,CAFA_ID_4,Entry_ID_2,Entry_ID_3,Entry_ID_4
5436,Q8BTM9,MNRKDIKRKSHQECSGKAGGRGRSRQARRHKTCPTPREISKVMASM...,10090,T100900005960,T100900005984,T100900005994,GRP4_MOUSE,GRP4_MOUSE,GRP4_MOUSE


### Run the Evaluation for CAFA4 H2H

In [40]:
BM_GO_path = "/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/"
pred_dir = "/data/yisupeng/sharing/cafa4/all_models/"
t_minus_ont_file = "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
IA_file = "/data/rashika/CAFA4/eval_final/IA.txt"
result_path = "/data/rashika/CAFA4/head2head/eval/cafa4_eval/"
#run_eval(BM_GO_path, pred_dir, t_minus_ont_file, IA_file, result_path, log_path = '/data/rashika/CAFA4/head2head/eval/cafa4_log/', thresh_step = 0.01)

Evaluating: bpo_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/yisupeng/sharing/cafa4/all_models/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/bpo_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa4_eval/bpo_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop max -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa4_log/bpo_all_type1/run.log &
Evaluating: cco_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/yisupeng/sharing/cafa4/all_models/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/cco_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa4_eval/cco_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop max -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa4_log/cco_all_type1/run.log &
Evaluating: mfo_all_type1.txt
python3 /home/

2024-06-30 00:31:58,167 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,246 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,358 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,363 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,379 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,432 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,572 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,597 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-06-30 00:31:58,696 [INFO ] Ontology: molecu

2024-06-30 00:33:21,144 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, evaluated
2024-06-30 00:33:23,819 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M118, evaluated
2024-06-30 00:33:24,043 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, cellular_component, proteins 1875, annotations 122254, replaced alt. ids 0
2024-06-30 00:33:41,937 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, evaluated
2024-06-30 00:33:43,617 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M118, evaluated
2024-06-30 00:33:47,815 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, molecular_function, proteins 1641, annotations 174991, replaced alt. ids 0
2024-06-30 00:33:48,630 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M046, cellular_component, proteins 901, annotations 94785, replaced alt. ids 1905
2024-06-30 00:33:50,826 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M046, cellular_compo

2024-06-30 00:37:41,248 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M065, evaluated
2024-06-30 00:37:44,075 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M077, cellular_component, proteins 733, annotations 8176, replaced alt. ids 0
2024-06-30 00:37:49,511 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M046, molecular_function, proteins 3806, annotations 314576, replaced alt. ids 14612
2024-06-30 00:37:55,033 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M077, evaluated
2024-06-30 00:37:57,041 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M031, cellular_component, proteins 566, annotations 4092, replaced alt. ids 0
2024-06-30 00:37:58,988 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M077, evaluated
2024-06-30 00:38:00,823 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M031, cellular_component, proteins 625, annotations 4514, replaced alt. ids 0
2024-06-30 00:38:02,018 [INFO ] Prediction: /

2024-06-30 00:42:02,632 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M045, cellular_component, proteins 927, annotations 157381, replaced alt. ids 0
2024-06-30 00:42:05,247 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M086, cellular_component, proteins 1670, annotations 36467, replaced alt. ids 0
2024-06-30 00:42:15,568 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M045, evaluated
2024-06-30 00:42:17,404 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M045, cellular_component, proteins 944, annotations 149560, replaced alt. ids 0
2024-06-30 00:42:29,411 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M045, evaluated
2024-06-30 00:42:33,284 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M086, evaluated
2024-06-30 00:42:47,735 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M056, molecular_function, proteins 2228, annotations 1122193, replaced alt. ids 0
2024-06-30 00:43:00,661 [INFO ] Prediction

...

2024-06-30 06:01:02,969 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M030, evaluated
2024-06-30 06:02:14,761 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M053, evaluated
2024-06-30 06:02:20,475 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M059, cellular_component, proteins 13004, annotations 5357940, replaced alt. ids 60502
2024-06-30 06:02:33,214 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M126, evaluated
2024-06-30 06:02:37,128 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M054, biological_process, proteins 2266, annotations 813590, replaced alt. ids 0
2024-06-30 06:03:22,979 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M082, molecular_function, proteins 9908, annotations 900269, replaced alt. ids 43676
2024-06-30 06:03:40,233 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M045, biological_process, proteins 19339, annotations 23234410, replaced alt. ids 0
2024-06-30 06:04:07,083 [IN

2024-06-30 06:27:47,150 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M036, evaluated
2024-06-30 06:28:00,234 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M136, biological_process, proteins 1282, annotations 37385, replaced alt. ids 0
2024-06-30 06:28:32,636 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M144, evaluated
2024-06-30 06:29:14,623 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/blast, cellular_component, proteins 12960, annotations 1571196, replaced alt. ids 144
2024-06-30 06:29:53,356 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M136, evaluated
2024-06-30 06:30:09,159 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M063, biological_process, proteins 1632, annotations 52223, replaced alt. ids 0
2024-06-30 06:30:58,951 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M023, evaluated
2024-06-30 06:31:00,816 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/blast, evaluated
2

2024-06-30 07:36:25,384 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M068, molecular_function, proteins 9830, annotations 231304, replaced alt. ids 0
2024-06-30 07:40:29,353 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M068, evaluated
2024-06-30 07:40:44,053 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M055, molecular_function, proteins 9908, annotations 889160, replaced alt. ids 0
2024-06-30 07:44:48,145 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M055, evaluated
2024-06-30 07:44:58,828 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M111, molecular_function, proteins 9861, annotations 212385, replaced alt. ids 0
2024-06-30 07:48:52,564 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M111, evaluated
2024-06-30 07:48:59,544 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M025, molecular_function, proteins 9905, annotations 187168, replaced alt. ids 0
2024-06-30 07:51:25,315 [INFO ] Predicti

2024-06-30 09:40:18,871 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M074, molecular_function, proteins 9, annotations 598, replaced alt. ids 0
2024-06-30 09:42:30,216 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M064, evaluated
2024-06-30 09:44:05,847 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M027, biological_process, proteins 19375, annotations 2068781, replaced alt. ids 0
2024-06-30 09:44:44,422 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M074, evaluated
2024-06-30 09:45:06,339 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M105, molecular_function, proteins 9716, annotations 367346, replaced alt. ids 0
2024-06-30 09:49:55,620 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M105, evaluated
2024-06-30 09:50:09,983 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M128, molecular_function, proteins 9157, annotations 224743, replaced alt. ids 0
2024-06-30 09:54:23,126 [INFO ] Prediction: 

2024-06-30 11:41:43,125 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M136, molecular_function, proteins 8079, annotations 125329, replaced alt. ids 0
2024-06-30 11:46:15,425 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M136, evaluated
2024-06-30 11:46:23,109 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M063, molecular_function, proteins 8406, annotations 98228, replaced alt. ids 0
2024-06-30 11:49:47,605 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M106, evaluated
2024-06-30 11:50:32,011 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M063, evaluated
2024-06-30 11:50:38,683 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M109, molecular_function, proteins 6962, annotations 102306, replaced alt. ids 0
2024-06-30 11:51:09,517 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M107, biological_process, proteins 14794, annotations 778651, replaced alt. ids 0
2024-06-30 11:54:46,832 [INFO ] Predicti

## Run for CAFA2 H2H

In [ ]:
BM_GO_path = "/data/rashika/CAFA4/head2head/eval/bm_GO_cafa2/"
pred_dir = "/data/yisupeng/sharing/cafa2/consolidated/"
t_minus_ont_file = "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
IA_file = "/data/rashika/CAFA4/eval_final/IA.txt"
result_path = "/data/rashika/CAFA4/head2head/eval/cafa2_eval/"
run_eval(BM_GO_path, pred_dir, t_minus_ont_file, IA_file, result_path, log_path = '/data/rashika/CAFA4/head2head/eval/cafa2_log/', thresh_step = 0.01)